# Create Neo4j Database

### Retrieve Required Data

In [1]:
import pandas as pd
import pickle

In [2]:
# User comedian matrix
matrix_filename = './user_comedian_matrix_OriginalPolarity.pkl'
ucm_df = pd.read_pickle(matrix_filename)
ucm_df.head()

,Ramy Youssef,Andy Woodhull,Amy Schumer,Arsenio Hall,Aziz Ansari,Bert Kreischer,Bill Burr,Chris D'Elia,Chris Rock,Dave Chappelle,...,Ronny Chieng,Roy Wood Jr.,Russell Peters,Sebastian Maniscalco,Tiffany Haddish,Tom Segura,Trevor Noah,Urzila Carlson,Vir Das,Whitney Cummings
A B,0.191429,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN
ADEDOTUN AKANDE,NaN,0.875,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,0.4,NaN,-0.1625,NaN,NaN,NaN,NaN,NaN,NaN
Anton Nym,NaN,0.68099,NaN,NaN,NaN,0,NaN,NaN,0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Lydia Waweru-Morgan,NaN,-0.2375,NaN,0.65,NaN,0.488281,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Jennifer Bates,NaN,0.671875,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [3]:
# Comedian Properties
cp_df = pd.read_excel('ComedianProperties.xlsx')
# Categorize the year of birth of the comedians
cp_df['year group'] = cp_df.year.apply(lambda x: str(x)[:3]+'0s')
# Remove unnecessary columns
cp_df = cp_df.drop(['spouseLabel','children_number','birthdate','year'], axis=1)
# change column names
cp_df.columns = ['Comedian','Citizenship','Ethnicity','Married','Gender','Year']
# Set index to comedian
cp_df = cp_df.set_index(cp_df['Comedian'].apply(lambda x:x.lower().strip()))
cp_df.head()

,Comedian,Citizenship,Ethnicity,Married,Gender,Year
Comedian,,,,,,
andy woodhull,Andy Woodhull,United States of America,White,True,male,1980s
ramy youssef,Ramy Youssef,United States of America,Egyptian-American Muslim,False,male,1990s
amy schumer,Amy Schumer,United States of America,American Jews,True,female,1980s
arsenio hall,Arsenio Hall,United States of America,African Americans,False,male,1950s
aziz ansari,Aziz Ansari,United States of America,Tamil American,False,male,1980s


In [4]:
# Comedian & keywords
kw_df = pd.read_excel('keywords_noun.xlsx',index_col=0)
kw_df.head()

,Comedian,Keywords
0,amy schumer,"thats thing, shes, kind, theyre thank, woman, ..."
1,andy woodhull,"im, thats, time wife wa, home, youre, dont, da..."
2,aziz ansari,"guy, thing, im, ims, youre, time, cause, kid, ..."
3,bert kreischer,"im, wife, shes, youre, fuck, fucking, dad, wa,..."
4,bill burr,"di, im, fucking, right, youre, person fuck, la..."


In [5]:
# Users and Comedians 
users = ucm_df.index
comedians = list(map(str.lower, ucm_df.columns))

### Build the graph in neo4j

In [15]:
from py2neo import Graph, Node, Relationship

In [22]:
host = "localhost"
password = "0000"
graph = Graph(host=host, password=password)

In [23]:
# Delete all nodes and relationships
DELETE_QUERY = "MATCH (n) DETACH DELETE n"
_ = graph.run(DELETE_QUERY)

In [24]:
# Add all user nodes
for user in users:
    graph.run("""
    MERGE (:User{name: $user})
    """, user=user)

In [25]:
# Add all comedian nodes
for comedian in comedians:
    properties = cp_df.loc[comedian]
    graph.run("""
    MERGE (:Comedian{name: $comedian, 
                      ethnicity: $ethnicity,
                      married: $married,
                      gender: $gender,
                      year: $year})
        """,comedian=comedian,
            citizenship=properties['Citizenship'],
            ethnicity=properties['Ethnicity'],
            married=int(properties['Married']),
            gender=properties['Gender'],
            year=properties['Year'])

In [26]:
# Build relationships for users and comedians
for c in ucm_df.columns:
    for u in ucm_df[c].keys():  
        rating = ucm_df.loc[u, c]
        if rating != 'NaN':
            graph.run("""
            MATCH (u:User{name: $user})
            MATCH (c:Comedian{name: $comedian})
            MERGE (u)-[r:commented_on{rating: $rr}]->(c)""", user=u, comedian=c.lower(), rr=rating)

In [27]:
# Add all keyword nodes
for i, r in kw_df.iterrows():
    comedian = r.Comedian
    keywords = r.Keywords.split(',')
    for word in keywords:
        graph.run("""
        MERGE (k:Keyword{word: $word})
        """, word=word)
        graph.run("""
        MATCH (c:Comedian{name: $comedian})
        MATCH (k:Keyword{word: $word})
        MERGE (c)-[:has_keyword]->(k)""", comedian=comedian, word=word)

In [ ]:
#